In [ ]:
from manipulator_casadi_model import fixedBaseManipulatorCasadiModel
from manipulator_acados_solver import ManipulatorSolverAcados
from utils import compute_5th_order_poly_traj
import matplotlib.pyplot as plt
import meshcat.geometry as g
import conf_kuka as conf
import pinocchio as pin
import casadi as ca
import numpy as np
import meshcat
import utils

In [ ]:
# create an end-effector trajectory using a quintic polynomial
x_ref = np.reshape(conf.ee_target_pos, (1, 3))
x_ref_N = np.repeat(x_ref, conf.N_traj, axis=0)
T = conf.N_traj*conf.dt
x, xdot, _ = compute_5th_order_poly_traj(conf.ee_init_pos, conf.ee_target_pos, T, conf.dt)
ee_ref = np.concatenate([x, xdot], axis=0)

In [ ]:
# create nominal NOCP 
nominal_solver = ManipulatorSolverAcados(
        fixedBaseManipulatorCasadiModel(conf, STOCHASTIC_OCP=False), ee_ref, MPC=False
        )
# solve nominal NOCP
X_nom, U_nom = nominal_solver.solve()

In [ ]:
# create stochastic NOCP (95 % obstacle avoidance constraint satisfaction)
stochastic_solver_95 = ManipulatorSolverAcados(
        fixedBaseManipulatorCasadiModel(conf, STOCHASTIC_OCP=True, epsilon=0.05), ee_ref, MPC=False
        )

# solve stochastic NOCP
X_stoch_95, U_stoch_95 = stochastic_solver_95.solve()

In [ ]:
# create stochastic NOCP (99 % obstacle avoidance constraint satisfaction) 
stochastic_solver_99 = ManipulatorSolverAcados(
        fixedBaseManipulatorCasadiModel(conf, STOCHASTIC_OCP=True, epsilon=0.01), ee_ref, MPC=False
        )

# solve stochastic NOCP
X_stoch_99, U_stoch_99 = stochastic_solver_99.solve()

In [ ]:
# visualize trajectories
dt = conf.dt
dt_ctrl = 0.01
N_ctrl =  int(dt/dt_ctrl)
rmodel = conf.rmodel
rdata = conf.rdata
ee_nom = np.zeros((3, N_ctrl*(X_nom.shape[0]-1))).astype(np.float32)
ee_stoch_95 = np.zeros((3, N_ctrl*(X_stoch_95.shape[0]-1))).astype(np.float32)
ee_stoch_99 = np.zeros((3, N_ctrl*(X_stoch_99.shape[0]-1))).astype(np.float32)
robot = conf.robot
# initialize meshcat viewer
viz = pin.visualize.MeshcatVisualizer(
        robot.model, robot.collision_model, robot.visual_model
        )
viz.initViewer(open=True)
viz.loadViewerModel()
# add L-shaped obstacle
utils.addViewerBox(viz, 'world/box1', .1, .1, .0, [1., .2, .2, .5])
utils.applyViewerConfiguration(viz, 'world/box1', [0.65, -0., 0.4, 1, 0, 0, 0])
utils.addViewerBox(viz, 'world/box2', .1, .0, .1, [1., .2, .2, .5])
utils.applyViewerConfiguration(viz, 'world/box2', [0.65, -0.05, 0.4+0.05, 1, 0, 0, 0])
# visualize nominal motion
for i in range(conf.N_traj-1):
    x_des_nom, tau_des_nom = nominal_solver.interpolate_one_step(
        X_nom[i, :7], X_nom[i+1, :7], 
        X_nom[i, 7:14], X_nom[i+1, 7:14],
        U_nom[i], U_nom[i+1]
    )
    for t in range(N_ctrl):
        q_des_nom = x_des_nom[t, :7]
        pin.framesForwardKinematics(rmodel, rdata, q_des_nom)
        ee_pos_nom = rdata.oMf[rmodel.getFrameId('contact')].translation
        ee_nom[:, i+t] = ee_pos_nom
        viz.display(q_des_nom)
# visualize stochastic motion (95 % obstacle avoidance constraint satisfaction)
for i in range(conf.N_traj-1):
    x_des_stoch_95, tau_des_stoch_95 = stochastic_solver_95.interpolate_one_step(
        X_stoch_95[i, :7], X_stoch_95[i+1, :7], 
        X_stoch_95[i, 7:14], X_stoch_95[i+1, 7:14],
        U_stoch_95[i], U_stoch_95[i+1]
    )
    for t in range(N_ctrl):
        q_des_stoch_95 = x_des_stoch_95[t, :7]
        pin.framesForwardKinematics(rmodel, rdata, q_des_stoch_95)
        ee_pos_stoch_95 = rdata.oMf[rmodel.getFrameId('contact')].translation
        ee_stoch_95[:, i+t] = ee_pos_stoch_95
        viz.display(q_des_stoch_95)
# visualize stochastic motion (99 % obstacle avoidance constraint satisfaction)
for i in range(conf.N_traj-1):
    x_des_stoch_99, tau_des_stoch_99 = stochastic_solver_99.interpolate_one_step(
        X_stoch_99[i, :7], X_stoch_99[i+1, :7], 
        X_stoch_99[i, 7:14], X_stoch_99[i+1, 7:14],
        U_stoch_99[i], U_stoch_99[i+1]
    )
    for t in range(N_ctrl):
        q_des_stoch_99 = x_des_stoch_99[t, :7]
        pin.framesForwardKinematics(rmodel, rdata, q_des_stoch_99)
        ee_pos_stoch_99 = rdata.oMf[rmodel.getFrameId('contact')].translation
        ee_stoch_99[:, i+t] = ee_pos_stoch_99
        viz.display(q_des_stoch_99)             
# display end-effector trajectory
utils.addLineSegment(viz, 'ee_trajectory_nom', ee_nom, [1,0,0,1])
utils.addLineSegment(viz, 'ee_trajectory_stoch_95', ee_stoch_95, [0,1,0,1])
utils.addLineSegment(viz, 'ee_trajectory_stoch_99', ee_stoch_99, [0,0,1,1])